# TREE - Arquitectura TreeEntry + Tree

Este notebook explora la **arquitectura modular** del sistema Tree.

## 🎯 Objetivo: Entender la estructura (sin calibración)

### 🌳 Concepto de ROOT (importante)

**ROOT = Referencia absoluta del laboratorio**

En tu sistema:
- **Set 57 (R3)** es el ROOT
- Contiene la referencia patrón del laboratorio
- **Todos los caminos terminan aquí**
- No tiene raised (no necesita calibrarse)

```
Jerarquía de calibración:

Set 57 (R3) ← ROOT ★ (referencia absoluta del lab)
    ↑
Set 49 (R2) ← Intermedia
    ↑
Set 3,4,5... (R1) ← Base (sensores a calibrar)

Flujo: R1 → R2 → R3 (ROOT)
```

---

### Componentes principales:

1. **TreeEntry** (Nodos del árbol):
   - Representa UN CalibSet con sus relaciones
   - Almacena: sensores, raised, parent/child links, offsets_to_raised
   - Es un NODO en el árbol

2. **Tree** (Contenedor jerárquico):
   - Organiza TODOS los TreeEntry
   - Estructura: R3 (ROOT) → R2 → R1
   - Es el ÁRBOL completo que une todos los nodos

### 📊 Analogía:
```
TreeEntry = Página individual de un libro (con links a otras páginas)
Tree = El libro completo que organiza todas las páginas
ROOT = La página de referencia principal (Set 57)
```

### Diferencia con TREE_CALIBRATION.ipynb:
- **Este notebook (TREE)**: Muestra la arquitectura y estructura
- **TREE_CALIBRATION**: Usa la estructura para CALCULAR constantes

---

## Conceptos clave:

- **offsets_to_raised**: Cada TreeEntry almacena offsets de sus sensores hacia cada raised
  - Formato: `{raised_id: {sensor_id: (offset, error)}}`
  - Permite múltiples caminos de calibración
  - **Incluye** offsets entre raised (ej: 48203 → 48479)
  - **NO incluye** offset consigo mismo (siempre 0)

- **Jerarquía bidireccional**: Cada TreeEntry conoce sus parents Y children
  - `parent_entries`: TreeEntries de la ronda anterior (hacia ROOT)
  - `children_entries`: TreeEntries de la ronda siguiente (lejos de ROOT)

## 1. Setup e Imports

In [5]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Añadir src al path
project_root = Path.cwd().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Imports del proyecto
from tree_entry import TreeEntry
from tree import Tree
from calibset import CalibSet
from logfile import Logfile
from utils.config import load_config
from utils.set_utils import create_calibration_set
from utils.tree_utils import create_tree_from_calibsets

print("[OK] Imports completados")
print(f"  Project root: {project_root}")
print(f"  Src path: {src_path}")

[OK] Imports completados
  Project root: /Users/vicky/Desktop/rtd-calib-desde0/RTD_Calibration
  Src path: /Users/vicky/Desktop/rtd-calib-desde0/RTD_Calibration/src


## 2. Cargar Configuración

In [6]:
# Cargar config.yml
config_path = project_root / 'config' / 'config.yml'
config = load_config(str(config_path))

print("[OK] Configuración cargada:")
print(f"  Total sets en config: {len(config['sensors']['sets'])}")
print(f"  Referencias generales: {config['sensors'].get('general_references', [])}")

# Ver ejemplo de un set
set_3 = config['sensors']['sets'].get(3.0, {})
print(f"\n  Ejemplo Set 3:")
print(f"    Sensors: {set_3.get('sensors', [])[:3]}...")
print(f"    Parent set: {set_3.get('parent_set', None)}")
print(f"    Discarded: {set_3.get('discarded', [])}")

[OK] Configuración cargada:
  Total sets en config: 60
  Referencias generales: []

  Ejemplo Set 3:
    Sensors: [48060, 48061, 48062]...
    Parent set: 49.0
    Discarded: [48205, 48478]


## 2.1 Crear un Tree Vacío

El Tree se puede crear vacío y luego rellenar con TreeEntries.

In [7]:
# Crear Tree vacío
empty_tree = Tree()

print("Tree vacío creado:")
print(f"  Entries: {len(empty_tree.entries)}")
print(f"  Root: {empty_tree.root}")
# get_entries_by_round() es un método, no un atributo
print(f"  Entries R1: {len(empty_tree.get_entries_by_round(1))}")
print(f"  Entries R2: {len(empty_tree.get_entries_by_round(2))}")
print(f"  Entries R3: {len(empty_tree.get_entries_by_round(3))}")

print("\n✓ El Tree está listo para recibir TreeEntries")
print("  Podemos añadirlos con tree.add_entry()")
print("  Y establecer el root con tree.set_root()")
print("\n[NOTA] Las rondas se calculan dinámicamente por distancia al root:")
print("  Root (Set 57) → R3")
print("  Hijos del root → R2")
print("  Nietos del root → R1")


Tree vacío creado:
  Entries: 0
  Root: None
  Entries R1: 0
  Entries R2: 0
  Entries R3: 0

✓ El Tree está listo para recibir TreeEntries
  Podemos añadirlos con tree.add_entry()
  Y establecer el root con tree.set_root()

[NOTA] Las rondas se calculan dinámicamente por distancia al root:
  Root (Set 57) → R3
  Hijos del root → R2
  Nietos del root → R1


## 2.2 Crear TreeEntry Manualmente y Añadirlo al Tree

Demostración de cómo crear un TreeEntry desde un CalibSet y añadirlo al Tree vacío.

In [8]:
# Cargar logfile
logfile_relative = config['paths']['logfile']
if logfile_relative.startswith('RTD_Calibration/'):
    logfile_relative = logfile_relative.replace('RTD_Calibration/', '', 1)
logfile_path = project_root / logfile_relative

logfile_obj = Logfile(filepath=str(logfile_path))
logfile_df = logfile_obj.log_file
print(f"✓ Logfile cargado: {len(logfile_df)} entradas")

# Crear UN CalibSet (Set 57 - el root)
# create_calibration_set devuelve (calibset, mean_offsets, std_offsets)
print("\nCreando CalibSet 57.0 (será el root)...")
calibset_57, _, _ = create_calibration_set(
    set_number=57.0,
    logfile=logfile_df,
    config=config
)
print(f"✓ CalibSet creado: {calibset_57}")

# Obtener info del config
set_57_config = config['sensors']['sets'][57.0]
discarded_ids_57 = set_57_config.get('discarded', [])

# discarded_sensors debe ser lista de objetos Sensor (no IDs)
discarded_sensors_57 = [s for s in calibset_57.sensors if s.id in discarded_ids_57]

# Crear TreeEntry manualmente
# TreeEntry es un dataclass: solo acepta calibset, discarded_sensors, raised_sensors,
# parent_entries, children_entries, offsets_to_raised
# set_number, round y sensors NO son parámetros (vienen de calibset)
tree_entry_57 = TreeEntry(
    calibset=calibset_57,
    discarded_sensors=discarded_sensors_57,
    raised_sensors=[],       # El root no tiene raised
    parent_entries=[],       # El root no tiene parents
    children_entries=[],     # Los añadiremos después
    offsets_to_raised={}     # Se calculará después
)

print(f"\n✓ TreeEntry creado:")
print(f"  Set: {tree_entry_57.calibset.set_number}")        # set_number viene del CalibSet
print(f"  Sensors: {len(tree_entry_57.calibset.sensors)}")  # sensors viene del CalibSet
print(f"  Raised: {tree_entry_57.raised_sensors}")
print(f"  Descartados: {[s.id for s in tree_entry_57.discarded_sensors]}")

# Añadir al Tree vacío
empty_tree.add_entry(tree_entry_57)
empty_tree.set_root(tree_entry_57)

print(f"\n✓ TreeEntry añadido al Tree:")
print(f"  Total entries: {len(empty_tree.entries)}")
print(f"  Root: Set {empty_tree.root.calibset.set_number if empty_tree.root else 'N/A'}")
# La ronda se calcula dinámicamente por distancia al root (BFS)
print(f"  Ronda del root: R{empty_tree.get_round(tree_entry_57)}")


CSV file loaded successfully from '/Users/vicky/Desktop/rtd-calib-desde0/RTD_Calibration/data/LogFile.csv'.
✓ Logfile cargado: 832 entradas

Creando CalibSet 57.0 (será el root)...
[OK] Set 57.0: 12 sensores creados
  Referencia: 48484 (canal 1)
  Procesando 9 runs válidos...
  [OK] Cargado 20250918_ln2_r48177_r48421_48484-55221_1: 695 registros, 14 canales
  [OK] Cargado 20250918_ln2_r48177_r48421_48484-55221_2: 683 registros, 14 canales
  [OK] Cargado 20250918_ln2_r48177_r48421_48484-55221_3: 684 registros, 14 canales
  [OK] Cargado 20250918_ln2_r48177_r48421_48484-55221_4: 824 registros, 14 canales
  [OK] Cargado 20250919_ln2_r48177_r48421_48484-55221_5: 698 registros, 14 canales
  [OK] Cargado 20250919_ln2_r48177_r48421_48484-55221_6: 685 registros, 14 canales
  [OK] Cargado 20251002_ln2_r48177_r48421_48484-55221_7: 705 registros, 14 canales
  [OK] Cargado 20251002_ln2_r48177_r48421_48484-55221_8: 699 registros, 14 canales
  [OK] Cargado 20251002_ln2_r48177_r48421_48484-55221_9: 71

## 2.3 Visualizar Tree con print()

El método `__str__()` de Tree permite ver la estructura jerárquica completa.

In [9]:
print("Estructura ACTUAL del Tree (solo con root):")
print("="*60)
print(empty_tree)
print("="*60)

print("\nInterpretación:")
print("  - El Tree tiene 1 entry (Set 57)")
print("  - Es la root (R3)")
print("  - No tiene parent ni children aún")

# Mostrar esqueleto completo que DEBERÍA tener
print("\n" + "="*60)
print("ESQUELETO COMPLETO del Tree (estructura esperada):")
print("="*60)

# Leer todos los sets del config y organizarlos por ronda
r1_sets = []
r2_sets = []
r3_sets = []

for set_id, set_info in config['sensors']['sets'].items():
    try:
        round_num = int(set_info['round'])
        if round_num == 1:
            r1_sets.append(set_id)
        elif round_num == 2:
            r2_sets.append(set_id)
        elif round_num == 3:
            r3_sets.append(set_id)
    except (ValueError, KeyError):
        continue

r1_sets = sorted(r1_sets)
r2_sets = sorted(r2_sets)
r3_sets = sorted(r3_sets)

# Mostrar estructura jerárquica esperada
print(f"\nR3 (Root/Referencia) - {len(r3_sets)} sets:")
for set_id in r3_sets:
    set_info = config['sensors']['sets'][set_id]
    raised = set_info.get('raised', [])
    print(f"  Set {int(set_id)}: raised={raised}")

print(f"\n  ↓ (conexiones via raised)")

print(f"\nR2 (Intermedia) - {len(r2_sets)} sets:")
for set_id in r2_sets[:5]:  # Mostrar solo primeros 5
    set_info = config['sensors']['sets'][set_id]
    raised = set_info.get('raised', [])
    parent = set_info.get('parent_set', 'N/A')
    print(f"  Set {int(set_id)}: parent={parent}, raised={raised}")
if len(r2_sets) > 5:
    print(f"  ... y {len(r2_sets)-5} sets más")

print(f"\n  ↓ (conexiones via raised)")

print(f"\nR1 (Base/Sensores a calibrar) - {len(r1_sets)} sets:")
for set_id in r1_sets[:10]:  # Mostrar solo primeros 10
    set_info = config['sensors']['sets'][set_id]
    raised = set_info.get('raised', [])
    parent = set_info.get('parent_set', 'N/A')
    n_sensors = len(set_info.get('sensors', []))
    print(f"  Set {int(set_id)}: parent={parent}, {n_sensors} sensores, raised={raised}")
if len(r1_sets) > 10:
    print(f"  ... y {len(r1_sets)-10} sets más")

print("\n" + "="*60)
print(f"Total estructura esperada: {len(r3_sets)+len(r2_sets)+len(r1_sets)} TreeEntries")
print("  - Actual: 1 entry (solo root)")
print("  - Falta: crear CalibSets y TreeEntries para los demás sets")


Estructura ACTUAL del Tree (solo con root):
Tree(1 entries, root=57.0)

Interpretación:
  - El Tree tiene 1 entry (Set 57)
  - Es la root (R3)
  - No tiene parent ni children aún

ESQUELETO COMPLETO del Tree (estructura esperada):

R3 (Root/Referencia) - 2 sets:
  Set 57: raised=[]
  Set 62: raised=[]

  ↓ (conexiones via raised)

R2 (Intermedia) - 8 sets:
  Set 49: parent=57.0, raised=[48484, 48747]
  Set 50: parent=57.0, raised=[48869, 48956]
  Set 51: parent=57.0, raised=[49112, 49167]
  Set 52: parent=57.0, raised=[49233, 55073]
  Set 53: parent=57.0, raised=[55253, 55227]
  ... y 3 sets más

  ↓ (conexiones via raised)

R1 (Base/Sensores a calibrar) - 48 sets:
  Set 3: parent=49.0, 12 sensores, raised=[48203, 48479]
  Set 4: parent=49.0, 12 sensores, raised=[48484, 48491]
  Set 5: parent=49.0, 12 sensores, raised=[48673, 48800]
  Set 6: parent=49.0, 12 sensores, raised=[48731, 48747]
  Set 7: parent=49.0, 12 sensores, raised=[48753, 48839]
  Set 8: parent=49.0, 12 sensores, raised

## 3. Crear Tree Completo con create_tree_from_calibsets()

### 🌳 ¿Qué es el ROOT?

**ROOT = Referencia absoluta del sistema de calibración**

El ROOT es el set de **Ronda 3 (R3)** que contiene la **referencia absoluta** del laboratorio.
Todos los demás sets se calibran "hacia" este root.

**En tu caso: Set 57 (R3) es el ROOT** porque:
- Contiene la referencia absoluta del laboratorio (sensor patrón)
- Es el punto más alto de la jerarquía
- **No tiene raised** (no necesita calibrarse con nadie más)
- Todos los caminos de calibración terminan aquí

**Jerarquía**:
```
Set 57 (R3) ← ROOT (referencia absoluta del lab)
    ↑
Set 49 (R2) ← Intermedia
    ↑
Set 3,4,5... (R1) ← Base (sensores a calibrar)
```

**Ejemplo de camino de calibración**:
```
Sensor 48060 (R1) → Raised 48203 (R1) → Sensor 48203 en Set 49 (R2) 
                                       → Raised 48484 (R2) → Sensor 48484 en Set 57 (R3/ROOT)
                                                            → Referencia absoluta ✓
```

---

### 📝 Especificación de Sets

En la **celda siguiente** puedes especificar qué sets explorar modificando:

```python
SETS_TO_EXPLORE = [3.0, 4.0, 5.0, 49.0, 57.0]  # ← CAMBIA AQUÍ
ROOT_SET = 57.0  # ← ROOT: Set de R3 con referencia absoluta
```

**Ejemplos**:
- `[3.0, 21.0, 49.0, 57.0]` → Sets 3 y 21 (R1) + parent 49 (R2) + root 57 (R3)
- `[3.0, 4.0, 49.0, 57.0]` → Jerarquía completa R1 → R2 → R3

**Importante**: El ROOT_SET debe ser **R3** (Set 57 o 58) porque es la referencia absoluta.

---

### ⚙️ Funcionamiento

La función `create_tree_from_calibsets()`:
1. Crea TreeEntries para cada CalibSet
2. Calcula `offsets_to_raised` para cada entry
3. Construye jerarquía parent-child según el config
4. Establece el root (referencia absoluta)

In [10]:
SETS_TO_EXPLORE = [3.0, 4.0, 5.0, 49.0, 57.0]  # Sets de ejemplo
ROOT_SET = 57.0  # Set que será la root del Tree (R3)

print(f"📋 Configuración de sets a explorar:")
print(f"   Sets: {SETS_TO_EXPLORE}")
print(f"   Root: {ROOT_SET}")
print()

sys.path.insert(0, str(src_path / 'utils'))
from tree_utils import create_tree_from_calibsets

calibsets = {}

print("Creando CalibSets...")
for set_id in SETS_TO_EXPLORE:
    try:
        # create_calibration_set devuelve (calibset, mean_offsets, std_offsets)
        calibset, _, _ = create_calibration_set(
            set_number=set_id,
            logfile=logfile_df,
            config=config
        )
        calibsets[set_id] = calibset
        
        set_config = config['sensors']['sets'].get(set_id, {})
        round_num = set_config.get('round', '?')
        print(f"  ✓ Set {int(set_id)} (R{round_num}): {len(calibset.sensors)} sensores, {len(calibset.runs)} runs")
    except Exception as e:
        print(f"  ✗ Set {int(set_id)}: Error - {e}")

print(f"\n✓ Total CalibSets creados: {len(calibsets)}")

print(f"\nCreando Tree con root={ROOT_SET}...")
tree = create_tree_from_calibsets(
    calibsets=calibsets,
    config=config,
    root_set_id=ROOT_SET
)

print(f"\n✓ Tree creado:")
print(f"  Total entries: {len(tree.entries)}")
# set_number viene de calibset, no del TreeEntry directamente
print(f"  Root: Set {tree.root.calibset.set_number if tree.root else 'N/A'}")

print(f"\n{'='*60}")
print("ESTRUCTURA COMPLETA DEL TREE:")
print(f"{'='*60}")
print(tree)
print(f"{'='*60}")

print(f"\n💡 Tip: Puedes cambiar SETS_TO_EXPLORE y ROOT_SET al inicio de esta celda")
print(f"   para explorar diferentes combinaciones de sets.")


📋 Configuración de sets a explorar:
   Sets: [3.0, 4.0, 5.0, 49.0, 57.0]
   Root: 57.0

Creando CalibSets...
[OK] Set 3.0: 12 sensores creados
  Referencia: 48060 (canal 1)
  Procesando 4 runs válidos...
  [OK] Cargado 20220531_ln2_r48176_r48177_48060_48479_7: 821 registros, 14 canales
  [OK] Cargado 20220531_ln2_r48176_r48177_48060_48479_8: 2260 registros, 14 canales
  [OK] Cargado 20220531_ln2_r48176_r48177_48060_48479_9: 794 registros, 14 canales
  [OK] Cargado 20220531_ln2_r48176_r48177_48060_48479_10: 680 registros, 14 canales
  [OK] 4 runs válidos con offsets
  [OK] Estadísticas calculadas: 12/12 sensores
  ✓ Set 3 (R1): 12 sensores, 4 runs
[OK] Set 4.0: 12 sensores creados
  Referencia: 48480 (canal 1)
  Procesando 4 runs válidos...
  [OK] Cargado 20220607_ln2_r48176_r48177_48480-48491_1: 985 registros, 14 canales
  [OK] Cargado 20220607_ln2_r48176_r48177_48480-48491_2: 1286 registros, 14 canales
  [OK] Cargado 20220607_ln2_r48176_r48177_48480-48491_3: 1127 registros, 14 canales

## 4. Crear Tree desde CalibSets

Usamos `create_tree_from_calibsets()` para construir automáticamente el Tree completo.

In [11]:
# El Tree ya está creado en el punto 3
print("✓ Tree ya creado en el punto anterior")
print(f"  Total entries: {len(tree.entries)}")
print(f"  Root: Set {tree.root.calibset.set_number if tree.root else 'N/A'}")

print(f"\nDistribución por ronda:")
for r in [1, 2, 3]:
    entries = tree.get_entries_by_round(r)
    print(f"  R{r}: {len(entries)} entries")

print(f"\nEstructura:")
print(tree)


✓ Tree ya creado en el punto anterior
  Total entries: 5
  Root: Set 57.0

Distribución por ronda:
  R1: 3 entries
  R2: 1 entries
  R3: 1 entries

Estructura:
Tree(5 entries, root=57.0)


## 5. Explorar TreeEntry

Examinamos un TreeEntry de R1 para ver su contenido.

In [12]:
# Obtener TreeEntry del Set 3 (R1)
entry_3 = tree.get_entry(3.0)

if entry_3:
    # set_number y sensors vienen del CalibSet; round lo calcula el Tree por BFS
    set_num = entry_3.calibset.set_number
    round_num = tree.get_round(entry_3)
    sensors = entry_3.calibset.sensors

    print(f"TreeEntry Set {set_num} (Round {round_num})")
    print("=" * 60)
    print(f"\nSensores:")
    print(f"  Total: {len(sensors)}")
    sensor_ids = [s.id for s in sensors]
    print(f"  IDs: {sensor_ids[:5]}..." if len(sensor_ids) > 5 else f"  IDs: {sensor_ids}")
    
    print(f"\nRaised sensors:")
    raised_ids = [s.id for s in entry_3.raised_sensors]
    print(f"  {raised_ids}")
    print(f"    → Estos raised de Set 3 aparecen como sensores en Set 49")
    
    print(f"\nSensores descartados:")
    discarded_ids = [s.id for s in entry_3.discarded_sensors]
    print(f"  {discarded_ids if discarded_ids else 'Ninguno'}")
    
    set_3_config = config['sensors']['sets'][3.0]
    raised_config = set_3_config.get('raised', [])
    parent_set = set_3_config.get('parent_set', 'N/A')
    
    print(f"\nRelaciones jerárquicas:")
    print(f"  Parents (hacia referencia): {[p.calibset.set_number for p in entry_3.parent_entries]}")
    if entry_3.parent_entries:
        print(f"    → Set {int(parent_set)} (R2) es el parent de Set 3 (R1)")
        print(f"    → Conexión: raised {raised_config} de Set 3 → sensores en Set {int(parent_set)}")
    
    print(f"\n  Children (más lejos de referencia): {[c.calibset.set_number for c in entry_3.children_entries]}")
    if not entry_3.children_entries:
        print(f"    → Set 3 está en R1 (base), NO tiene children")
    
    print(f"\nOffsets to raised disponibles:")
    # offsets_to_raised keys son objetos Sensor, no IDs
    for raised_sensor, offsets_dict in entry_3.offsets_to_raised.items():
        print(f"  Raised {raised_sensor.id}: {len(offsets_dict)} sensores con offset calculado")
else:
    print("⚠️  Entry no encontrado")


TreeEntry Set 3.0 (Round 1)

Sensores:
  Total: 12
  IDs: [48060, 48061, 48062, 48063, 48202]...

Raised sensors:
  [48203, 48479]
    → Estos raised de Set 3 aparecen como sensores en Set 49

Sensores descartados:
  [48205, 48478]

Relaciones jerárquicas:
  Parents (hacia referencia): [49.0]
    → Set 49 (R2) es el parent de Set 3 (R1)
    → Conexión: raised [48203, 48479] de Set 3 → sensores en Set 49

  Children (más lejos de referencia): []
    → Set 3 está en R1 (base), NO tiene children

Offsets to raised disponibles:
  Raised 48203: 9 sensores con offset calculado
  Raised 48479: 9 sensores con offset calculado


## 6. Explorar Offsets to Raised

Veamos los offsets calculados para un sensor específico hacia diferentes raised.

In [13]:
if entry_3:
    valid_sensors = entry_3.get_valid_sensors()
    
    if valid_sensors:
        test_sensor = valid_sensors[0]  # Objeto Sensor
        
        print(f"Ejemplo: Offsets del Sensor {test_sensor.id}")
        print("=" * 60)
        
        # raised_sensors son objetos Sensor
        for raised_sensor in entry_3.raised_sensors:
            offset_data = entry_3.get_offset_to_raised(test_sensor, raised_sensor)
            
            if offset_data:
                offset, error = offset_data
                print(f"\nSensor {test_sensor.id} → Raised {raised_sensor.id}:")
                print(f"  Offset: {offset:+.6f} K")
                print(f"  Error:  {error:.6f} K")
                print(f"  SNR:    {abs(offset/error):.1f}" if error != 0 else "  SNR:    ∞")
            else:
                print(f"\nSensor {test_sensor.id} → Raised {raised_sensor.id}: No disponible")
        
        # Mostrar offsets ENTRE raised
        if len(entry_3.raised_sensors) > 1:
            print(f"\n{'='*60}")
            print(f"Offsets ENTRE RAISED:")
            print(f"{'='*60}")
            
            raised_list = sorted(entry_3.raised_sensors, key=lambda s: s.id)
            for i, r1 in enumerate(raised_list):
                for r2 in raised_list[i+1:]:
                    offset_data = entry_3.get_offset_to_raised(r2, r1)
                    if offset_data:
                        offset, error = offset_data
                        print(f"\nRaised {r1.id} → Raised {r2.id}:")
                        print(f"  Offset: {offset:+.6f} K")
                        print(f"  Error:  {error:.6f} K")
                        print(f"  → Permite caminos alternativos en calibración")
    else:
        print("⚠️  No hay sensores válidos en este entry")


Ejemplo: Offsets del Sensor 48060

Sensor 48060 → Raised 48203:
  Offset: +0.073624 K
  Error:  0.000183 K
  SNR:    401.5

Sensor 48060 → Raised 48479:
  Offset: +0.066355 K
  Error:  0.000176 K
  SNR:    377.8

Offsets ENTRE RAISED:

Raised 48203 → Raised 48479:
  Offset: +0.007269 K
  Error:  0.000254 K
  → Permite caminos alternativos en calibración


### 📊 ¿Por qué no 12 offsets?

**Respuesta rápida**: Porque hay sensores descartados y no se calcula el offset de un raised consigo mismo.

#### Ejemplo Set 3:
- **12 sensores totales**
- **-2 descartados** (configurados en `config.yml`) → NO se calculan
- **= 10 sensores válidos**

Para cada raised, se calculan offsets hacia **9 sensores**:
- Los 8 sensores normales
- El otro raised (permite caminos alternativos)
- **NO él mismo** (offset consigo mismo siempre sería 0, no aporta información)

**Matemática**: 10 válidos - 1 (él mismo) = 9 offsets por raised ✓

#### ❓ ¿Por qué NO guardar offset de un raised consigo mismo (0)?

**No aporta información útil**:
- `offset(48203 → 48203) = 0.0` siempre
- No tiene error asociado
- Desperdicia memoria
- Complica la lógica (habría que filtrarlo después)

**El código funciona perfectamente sin él**:
- Si necesitas saber si un sensor es raised, usa `entry.is_sensor_raised()`
- Los caminos de calibración nunca usan `sensor → él mismo`
- Los offsets entre DIFERENTES raised (ej: 48203 → 48479) sí están guardados

**Conclusión**: La exclusión de offset=0 es correcta y eficiente. ✓

In [14]:
if entry_3:
    print("RESUMEN: Set 3 - Offsets calculados")
    print("=" * 60)
    
    # Inventario básico — sensors/raised/discarded son listas de objetos Sensor
    all_sensors = set(entry_3.calibset.sensors)
    raised = set(entry_3.raised_sensors)
    discarded = set(entry_3.discarded_sensors)
    valid_sensors = all_sensors - discarded
    
    print(f"\n📋 Set {entry_3.calibset.set_number}:")
    print(f"   Total sensores:  {len(all_sensors)}")
    print(f"   Descartados:     {len(discarded)} → {sorted([s.id for s in discarded]) if discarded else 'Ninguno'}")
    print(f"   Raised:          {len(raised)} → {sorted([s.id for s in raised])}")
    print(f"   Válidos:         {len(valid_sensors)}")
    
    # Offsets por raised — keys son objetos Sensor
    print(f"\n🎯 Offsets calculados:")
    for raised_sensor, offsets_dict in entry_3.offsets_to_raised.items():
        print(f"   Raised {raised_sensor.id}: {len(offsets_dict)} offsets")
    
    print(f"\n   Fórmula: {len(valid_sensors)} válidos - 1 (él mismo) = {len(valid_sensors)-1} offsets ✓")
    
    # Conectividad entre raised
    if len(raised) > 1:
        print(f"\n🔗 Caminos entre raised:")
        raised_list = sorted(raised, key=lambda s: s.id)
        for i, r1 in enumerate(raised_list):
            for r2 in raised_list[i+1:]:
                # offsets_to_raised[r1][r2] — r2 es objeto Sensor
                if r2 in entry_3.offsets_to_raised.get(r1, {}):
                    offset, error = entry_3.offsets_to_raised[r1][r2]
                    print(f"   {r1.id} ↔ {r2.id}: {offset:+.6f} K ± {error:.6f} K")
                    print(f"      → Permite caminos alternativos de calibración")
else:
    print("⚠️  Entry no encontrado")


RESUMEN: Set 3 - Offsets calculados

📋 Set 3.0:
   Total sensores:  12
   Descartados:     2 → [48205, 48478]
   Raised:          2 → [48203, 48479]
   Válidos:         10

🎯 Offsets calculados:
   Raised 48203: 9 offsets
   Raised 48479: 9 offsets

   Fórmula: 10 válidos - 1 (él mismo) = 9 offsets ✓

🔗 Caminos entre raised:
   48203 ↔ 48479: +0.007269 K ± 0.000254 K
      → Permite caminos alternativos de calibración


## 7. Navegación por Rondas

El Tree organiza entries por rondas para facilitar el procesamiento.

In [15]:
print("Entries por ronda:")
print("=" * 60)

for round_num in [1, 2, 3]:
    entries = tree.get_entries_by_round(round_num)
    print(f"\nRonda {round_num}: {len(entries)} entries")
    
    if entries:
        # set_number viene del CalibSet
        set_numbers = sorted([e.calibset.set_number for e in entries])
        print(f"  Sets: {set_numbers}")
        
        # sensors/raised/discarded son listas de objetos Sensor
        total_sensors = sum(len(e.calibset.sensors) for e in entries)
        total_raised = sum(len(e.raised_sensors) for e in entries)
        total_discarded = sum(len(e.discarded_sensors) for e in entries)
        
        print(f"  Total sensores: {total_sensors}")
        print(f"  Total raised: {total_raised}")
        print(f"  Total descartados: {total_discarded}")


Entries por ronda:

Ronda 1: 3 entries
  Sets: [3.0, 4.0, 5.0]
  Total sensores: 36
  Total raised: 6
  Total descartados: 5

Ronda 2: 1 entries
  Sets: [49.0]
  Total sensores: 12
  Total raised: 2
  Total descartados: 0

Ronda 3: 1 entries
  Sets: [57.0]
  Total sensores: 12
  Total raised: 0
  Total descartados: 0


## 8. Visualización de Jerarquía

El método `__str__()` de Tree muestra la estructura jerárquica.

In [16]:
print("Estructura jerárquica del Tree:")
print("=" * 60)
print(tree)

print("\nLeyenda:")
print("  R3: Ronda 3 (Root/Referencia absoluta)")
print("  R2: Ronda 2 (Intermedia)")
print("  R1: Ronda 1 (Base - sensores a calibrar)")
print("\n  → : Relación parent → child")

Estructura jerárquica del Tree:
Tree(5 entries, root=57.0)

Leyenda:
  R3: Ronda 3 (Root/Referencia absoluta)
  R2: Ronda 2 (Intermedia)
  R1: Ronda 1 (Base - sensores a calibrar)

  → : Relación parent → child


## 9. Verificar Conectividad

Comprobamos que los enlaces parent-child son bidireccionales.

In [17]:
print("Verificación de conectividad bidireccional:")
print("=" * 60)

all_ok = True

for set_num, entry in tree.entries.items():
    # Verificar parents
    for parent in entry.parent_entries:
        if entry not in parent.children_entries:
            print(f"⚠️  Set {set_num}: parent {parent.set_number} no tiene a este entry como child")
            all_ok = False
    
    # Verificar children
    for child in entry.children_entries:
        if entry not in child.parent_entries:
            print(f"⚠️  Set {set_num}: child {child.set_number} no tiene a este entry como parent")
            all_ok = False

if all_ok:
    print("✓ Todos los enlaces son bidireccionales")
    print("✓ La estructura del Tree es consistente")

Verificación de conectividad bidireccional:
✓ Todos los enlaces son bidireccionales
✓ La estructura del Tree es consistente


## 10. Resumen de Arquitectura

### ✅ Lo que hemos aprendido:

1. **ROOT (Referencia absoluta)**:
   - Set 57 (R3) es el ROOT del sistema
   - Contiene la referencia absoluta del laboratorio
   - Todos los caminos de calibración terminan aquí
   - **No tiene raised** porque no necesita calibrarse con nadie

2. **TreeEntry** (Nodo individual):
   - Representa UN CalibSet con relaciones
   - Almacena: sensors, raised, parent/child links, offsets_to_raised
   - Es un NODO en el árbol

3. **Tree** (Contenedor completo):
   - Organiza TODOS los TreeEntry
   - Estructura jerárquica: R3 (ROOT) → R2 → R1
   - Se puede crear vacío con `Tree()` y rellenar con `add_entry()`
   - Visualizar con `print(tree)` muestra estructura completa

4. **Offsets to raised**:
   - Se calculan offsets de cada sensor hacia cada raised
   - **Incluye** offsets entre raised (ej: 48203 → 48479)
   - **NO incluye** offset de un raised consigo mismo (siempre sería 0)
   - Permite múltiples caminos de calibración

5. **Construcción**:
   ```python
   # Manual (paso a paso)
   tree = Tree()
   entry = TreeEntry(...)
   tree.add_entry(entry)
   tree.set_root(entry)
   
   # Automática (producción)
   tree = create_tree_from_calibsets(
       calibsets, 
       config, 
       root_set_id=57.0  # ROOT = R3
   )
   ```

6. **Visualización**:
   ```python
   print(tree)  # Muestra jerarquía completa desde ROOT
   ```

### 🔄 Diferencias entre notebooks:

| Aspecto | TREE.ipynb (ESTE) | TREE_CALIBRATION.ipynb |
|---------|-------------------|------------------------|
| Objetivo | Mostrar ARQUITECTURA | CALCULAR constantes |
| Enfoque | TreeEntry + Tree | Uso de Tree |
| Complejidad | Básico (5 sets) | Completo (60 sets) |
| Resultado | Entender estructura | Obtener constantes |

### 📚 Próximos pasos:

Ver **TREE_CALIBRATION.ipynb** para:
- Búsqueda de caminos R1 → R2 → R3 (ROOT)
- Media ponderada de múltiples caminos
- Cálculo de constantes finales

### 📖 Documentación adicional:

- `docs/NOTEBOOKS_GUIDE.md`: Guía completa de diferencias
- `docs/REFACTORING_SUMMARY.md`: Resumen de la nueva arquitectura

In [18]:
print("\n" + "=" * 60)
print("✓ Notebook TREE completado")
print("=" * 60)
print("\nArchivos de la arquitectura:")
print("  src/tree_entry.py        - Clase TreeEntry (169 líneas)")
print("  src/tree.py              - Clase Tree (66 líneas)")
print("  src/utils/tree_utils.py  - Construcción del Tree")
print("  src/utils/calibration_utils.py - Cálculo de constantes")
print("\nPróximo notebook:")
print("  notebooks/TREE_CALIBRATION.ipynb")


✓ Notebook TREE completado

Archivos de la arquitectura:
  src/tree_entry.py        - Clase TreeEntry (169 líneas)
  src/tree.py              - Clase Tree (66 líneas)
  src/utils/tree_utils.py  - Construcción del Tree
  src/utils/calibration_utils.py - Cálculo de constantes

Próximo notebook:
  notebooks/TREE_CALIBRATION.ipynb
